In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedShuffleSplit



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/charliedekock/Desktop/Machine_Learning/ml_env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/charliedekock/Desktop/Machine_Learning/ml_env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/charliedekock/Desktop/Machine_Learning/ml_env/lib/python3.11/site-package

In [4]:
data = pd.read_csv('../data/blob_data.csv')
data

y = data['y']
X = data.drop(columns='y')

### K-fold setup

In [13]:
def generate_folds(X, y, n_folds = 10):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=None)
    folds = []
    splits = kf.split(X, y)
    for train_index, val_index in splits:
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        folds.append({'Xt': X_train, 'Xv': X_val, 'yt': y_train, 'yv': y_val})
    return folds

In [14]:
folds = generate_folds(X, y, 10)

### Model Training functions

In [16]:
class FeedforwardNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.act = nn.ReLU()  
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        return x

In [29]:
def train_sgd_model(fold, hidden_dim=2, input_dim = 2, output_dim = 2, batch_size=1, learning_rate = 0.1, max_epochs=10, to_print=True): 
    X_val = torch.tensor(fold['Xv'].values, dtype=torch.float32)
    y_val = torch.tensor(fold['yv'].values, dtype=torch.long)
    X_train = torch.tensor(fold['Xt'].values, dtype=torch.float32)
    y_train = torch.tensor(fold['yt'].values, dtype=torch.long)

    train_dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    train_dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model = FeedforwardNet(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []
    for epoch in range(max_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        for batch_X, batch_y in loader:  # batch_size=1
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            total += 1
        
        avg_train_loss = train_loss / total
        train_losses.append(avg_train_loss)
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)              
            val_loss = criterion(val_outputs, y_val).item()  
            val_losses.append(val_loss)

        if to_print == True:
            print(f"Epoch {epoch+1}/{max_epochs} | "
                f"Train Loss: {avg_train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f}")

    return  

In [28]:
train_sgd_model(folds[0], max_epochs=1)

TypeError: 'bool' object is not callable